In [1]:
import pandas as pd
import string

In [2]:
chars = [i for i in string.ascii_uppercase + string.ascii_lowercase + string.digits + string.punctuation + ' ' + '\t' + '\n']

In [3]:
len(chars)

97

In [4]:
chars[:5]

['A', 'B', 'C', 'D', 'E']

In [5]:
chars_pos = lambda c: chars.index(c)

In [6]:
samples = pd.read_csv('data/dataset_random.csv', header=None)

In [7]:
num_samples = len(samples)

In [8]:
max_len_ptbr = len(max(samples[1][1:], key=len))
max_len_sql = len(max(samples[1][1:], key=len))

In [9]:
ptbr = samples[1][1:].tolist()
sql = samples[2][1:].tolist()

In [10]:
import numpy as np

In [11]:
ptbr_tokenized = np.zeros((num_samples, max_len_ptbr, len(chars)))
sql_tokenized = np.zeros((num_samples, max_len_sql, len(chars)))
target = np.zeros((num_samples, max_len_sql, len(chars)))

In [12]:
for i in range(len(samples)-1):
    for j, ch in enumerate(ptbr[i]):
        ptbr_tokenized[i, j, chars_pos(ch)] = 1
    
    for j, ch in enumerate(sql[i]):
        sql_tokenized[i, j, chars_pos(ch)] = 1
        
        if j > 0:
            target[i, j-1, chars_pos(ch)] = 1

In [13]:
import keras

from keras.layers import Input, LSTM, Dense
from keras.models import Model

Using TensorFlow backend.


In [14]:
e_input = Input((None, len(chars)))
e_lstm = LSTM(256, return_state=True)
e_output, h, c = e_lstm(e_input)
e_states = [h, c]
d_input = Input((None, len(chars)))
d_lstm = LSTM(256, return_sequences=True, return_state=True)
d_output, _, _ = d_lstm(d_input, initial_state=e_states)
d_dense = Dense(len(chars), activation='softmax')
d_output = d_dense(d_output)

In [15]:
model = Model(inputs=[e_input, d_input], outputs=[d_output])
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [16]:
model.fit(x=[ptbr_tokenized, sql_tokenized], y=target, batch_size=64, epochs=50, validation_split=0.2)

Train on 19200 samples, validate on 4801 samples
Epoch 1/50
19200/19200 [==============================] - 93s 5ms/step - loss: 1.0928 - val_loss: 1.2529
Epoch 2/50
19200/19200 [==============================] - 89s 5ms/step - loss: 0.9192 - val_loss: 1.2280
Epoch 3/50
19200/19200 [==============================] - 88s 5ms/step - loss: 0.9107 - val_loss: 1.2210
Epoch 4/50
19200/19200 [==============================] - 87s 5ms/step - loss: 0.9076 - val_loss: 1.2225
Epoch 5/50
19200/19200 [==============================] - 88s 5ms/step - loss: 0.9016 - val_loss: 1.2144
Epoch 6/50
19200/19200 [==============================] - 86s 5ms/step - loss: 0.9090 - val_loss: 1.2218
Epoch 7/50
19200/19200 [==============================] - 87s 5ms/step - loss: 0.9025 - val_loss: 1.2133
Epoch 8/50
19200/19200 [==============================] - 87s 5ms/step - loss: 0.9000 - val_loss: 1.2133
Epoch 9/50
19200/19200 [==============================] - 87s 5ms/step - loss: 0.8994 - val_loss: 1.2123
Epoch 

In [17]:
model.save('models/ptbr2sql_without_attention.h5')

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/keras/engine/network.py:888: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [18]:
e_inference_model = Model(e_input, e_states)

In [19]:
d_state_h = Input((256,))
d_state_c = Input((256,))
d_input_states = [d_state_h, d_state_c]

In [20]:
d_output, d_h, d_c = d_lstm(d_input, initial_state=d_input_states)
d_states = [d_h, d_c]
d_output = d_dense(d_output)

In [21]:
d_inference_model = Model(inputs=[d_input] + d_input_states, outputs=[d_output] + d_states)

In [22]:
def decode(s):
    states = e_inference_model.predict(s)
    
    target = np.zeros((1, 1, len(chars)))
    target[0, 0, chars_pos('\t')] = 1
    
    sql = ''
    stop = False
    
    while not stop:
        d_out, d_h, d_c = d_inference_model.predict(x=[target] + states)
        
        max_index = np.argmax(d_out[0, -1, :])
        sampled = chars[max_index]
        sql += str(sampled)
        
        if(sampled == '\n' or (len(sql) > max_len_sql)):
            stop = True
            
        target = np.zeros((1, 1, len(chars)))
        target[0, 0, max_index] = 1
        
        states = [d_h, d_c]
    
    return sql

In [24]:
for t in range(0, 20):
    s = ptbr_tokenized[t]
    s = np.expand_dims(s, axis=0)
    
    print(s.shape)
    
    translated = decode(s)
    
    print("Input:", ptbr[t])
    print("Output:", translated)

(1, 100, 97)
Input: traga tudo da tabela x1v3NLJGBFq1F6k
Output: select * from DzgggQ3GQlhhcZ

(1, 100, 97)
Input: traga tudo da tabela 1fnR
Output: select * from fU3l

(1, 100, 97)
Input: traga tudo da tabela EakxSAl
Output: select * from USUYlx

(1, 100, 97)
Input: traga tudo da tabela Hlt11CGmim
Output: select * from kptQm3Q3xa

(1, 100, 97)
Input: traga tudo da tabela RpKjOXdT9ln20q
Output: select * from krzJtg1Ohfx5Mx

(1, 100, 97)
Input: traga tudo da tabela cLZ71zvw
Output: select * from 4oEHvm3aa

(1, 100, 97)
Input: traga tudo da tabela YRFGeqn7ESC9Ue6
Output: select * from YFgggggggMNFnN

(1, 100, 97)
Input: traga tudo da tabela 1oOlRWvQxjhM
Output: select * from 4Or7Xmz8otA3

(1, 100, 97)
Input: traga tudo da tabela 7tnWRpC3no
Output: select * from YFgggglNB

(1, 100, 97)
Input: traga tudo da tabela o9C9ui
Output: select * from DFji7d

(1, 100, 97)
Input: traga tudo da tabela o
Output: select * from o

(1, 100, 97)
Input: traga tudo da tabela yuhKGT9
Output: select * from kK